[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/RF.ipynb)

# RandomForest(回帰モデル)

In [ ]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [2]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [4]:
# 評価値をユーザー×映画の行列に変換。欠損値は、平均値または０で穴埋めする
user_movie_matrix = movielens_train.pivot(index="user_id", columns="movie_id", values="rating")
user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))
user_movie_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,62000,62113,62293,62344,62394,62801,62803,63113,63992,64716
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,5.0,NaN,3.0,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 学習に用いる学習用データ中のユーザーと映画の組を取得する
train_keys = movielens_train[["user_id", "movie_id"]]
# 学習用データ中の評価値を学習の正解データとして取得する
train_y = movielens_train.rating.values

# 評価値を予測したいテスト用データ中のユーザーと映画の組を取得する
test_keys = movielens_test[["user_id", "movie_id"]]
# ランキング形式の推薦リスト作成のために学習用データに存在するすべてのユーザーとすべての映画の組み合わせを取得する
train_all_keys = user_movie_matrix.stack(dropna=False).reset_index()[["user_id", "movie_id"]]


In [7]:
# 特徴量を作成する
train_x = train_keys.copy()
test_x = test_keys.copy()
train_all_x = train_all_keys.copy()

In [9]:
# 学習用データに存在するユーザーごとの評価値の最小値、最大値、平均値
# 及び、映画ごとの評価値の最小値、最大値、平均値を特徴量として追加
aggregators = ["min", "max", "mean"]
user_features = movielens_train.groupby("user_id").rating.agg(aggregators).to_dict()
movie_features = movielens_train.groupby("movie_id").rating.agg(aggregators).to_dict()
for agg in aggregators:
    train_x[f"u_{agg}"] = train_x["user_id"].map(user_features[agg])
    test_x[f"u_{agg}"] = test_x["user_id"].map(user_features[agg])
    train_all_x[f"u_{agg}"] = train_all_x["user_id"].map(user_features[agg])
    train_x[f"m_{agg}"] = train_x["movie_id"].map(movie_features[agg])
    test_x[f"m_{agg}"] = test_x["movie_id"].map(movie_features[agg])
    train_all_x[f"m_{agg}"] = train_all_x["movie_id"].map(movie_features[agg])
# テスト用データにしか存在しないユーザーや映画の特徴量を、学習用データ全体の平均評価値で埋める
average_rating = train_y.mean()
test_x.fillna(average_rating, inplace=True)

In [ ]:
import itertools

# 映画が特定の genre であるかどうかを表す特徴量を追加
movie_genres = movies[["movie_id", "genre"]]
genres = set(list(itertools.chain(*movie_genres.genre)))
for genre in genres:
    movie_genres[f"is_{genre}"] = movie_genres.genre.apply(lambda x: genre in x)
movie_genres.drop("genre", axis=1, inplace=True)
train_x = train_x.merge(movie_genres, on="movie_id")
test_x = test_x.merge(movie_genres, on="movie_id")
train_all_x = train_all_x.merge(movie_genres, on="movie_id")

In [13]:
# 特徴量としては使わない情報を削除
train_x = train_x.drop(columns=["user_id", "movie_id"])
test_x = test_x.drop(columns=["user_id", "movie_id"])
train_all_x = train_all_x.drop(columns=["user_id", "movie_id"])

In [15]:
from sklearn.ensemble import RandomForestRegressor as RFR

# Random Forest を用いた学習
reg = RFR(n_jobs=-1, random_state=0)
reg.fit(train_x.values, train_y)


RandomForestRegressor(n_jobs=-1, random_state=0)

In [16]:
# テスト用データ内のユーザーと映画の組に対して評価値を予測する
test_pred = reg.predict(test_x.values)

movie_rating_predict = test_keys.copy()
movie_rating_predict["rating_pred"] = test_pred

In [20]:
from collections import defaultdict
import numpy as np

# 学習用データに存在するすべてのユーザーとすべての映画の組み合わせに対して評価値を予測する
train_all_pred = reg.predict(train_all_x.values)

pred_train_all = train_all_keys.copy()
pred_train_all["rating_pred"] = train_all_pred
pred_matrix = pred_train_all.pivot(index="user_id", columns="movie_id", values="rating_pred")

# ユーザーが学習用データ内で評価していない映画の中から
# 予測評価値が高い順に10件の映画をランキング形式の推薦リストとする
pred_user2items = defaultdict(list)
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
for user_id in movielens_train.user_id.unique():
    movie_indexes = np.argsort(-pred_matrix.loc[user_id, :]).values
    for movie_index in movie_indexes:
        movie_id = user_movie_matrix.columns[movie_index]
        if movie_id not in (user_evaluated_movies[user_id]):
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {139: [7986, 6565, 5817, 6356, 4828, 6538, 4270, 26974, 631, 3588],
             149: [5914, 640, 555, 1672, 1570, 5996, 5928, 1326, 6689, 719],
             182: [6729,
              4956,
              5971,
              8379,
              6713,
              30894,
              5662,
              6890,
              7647,
              5090],
             215: [5598, 4626, 4679, 6286, 6279, 848, 1600, 4458, 4802, 5572],
             281: [8482, 1497, 880, 4531, 715, 748, 6583, 5472, 1483, 1992],
             326: [2211, 1084, 2393, 2269, 1643, 1711, 1336, 1186, 1156, 2334],
             351: [6312, 6058, 7458, 7117, 7523, 6008, 7340, 7836, 8485, 8369],
             357: [56949, 59037, 13, 2569, 56587, 2362, 2142, 2013, 2243, 295],
             426: [3718, 3035, 7817, 3620, 3892, 3387, 5723, 3496, 3223, 5352],
             456: [5060, 5688, 5634, 4975, 6672, 5909, 5275, 4988, 6568, 1880],
             459: [6005,
              40851,
              33

In [21]:
# user_id=2のユーザーが学習データで評価を付けた映画一覧
movielens_train[movielens_train.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
6150,2,648,2.0,868244699,Mission: Impossible (1996),"[Action, Adventure, Mystery, Thriller]","[confusing, confusing plot, memorable sequence...",12.0
6531,2,733,3.0,868244562,"Rock, The (1996)","[Action, Adventure, Thriller]","[gfei own it, alcatraz, nicolas cage, sean con...",18.0
6813,2,736,3.0,868244698,Twister (1996),"[Action, Adventure, Romance, Thriller]","[disaster, disaster, storm, bill paxton, helen...",13.0
7113,2,780,3.0,868244698,Independence Day (a.k.a. ID4) (1996),"[Action, Adventure, Sci-Fi, War]","[action, alien invasion, aliens, will smith, a...",14.0
7506,2,786,3.0,868244562,Eraser (1996),"[Action, Drama, Thriller]","[arnold schwarzenegger, action, arnold, arnold...",19.0
7661,2,802,2.0,868244603,Phenomenon (1996),"[Drama, Romance]","[interesting concept, own, john travolta, john...",15.0
7779,2,858,2.0,868245645,"Godfather, The (1972)","[Crime, Drama]","[oscar (best picture), marlon brando, classic,...",9.0


In [22]:
pred_user2items[2]

[4210, 4961, 4105, 2351, 4032, 1586, 2804, 5159, 4840, 3350]

In [23]:
# user_id=2に対するおすすめ(4210, 4961, 4105)
movies[movies.movie_id.isin([4210, 4961, 4105])]

,movie_id,title,genre,tag
4013,4105,"Evil Dead, The (1981)","[Fantasy, Horror]","[directorial debut, bruce campbell, cult class..."
4118,4210,Manhunter (1986),"[Action, Crime, Drama, Horror, Thriller]","[hannibal lecter, serial killer, ei muista, er..."
4867,4961,Pornstar: The Legend of Ron Jeremy (2001),[Documentary],[pornography]
